In [151]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

import plotly.graph_objects as go
import plotly.express as px


In [112]:
colors = [
    "#efc86e",
    "#6f9969",
    "#97c684",
    "#aab5d5",
    "#808fe1",
    "#5c66a8",
    "#454a74",
]

In [101]:
df = pd.read_csv('Prostate_Cancer_TFM/Files/Own_data/BIMCV_with_predictions.csv')

# Adjust the probability based on the prediction value
df['adjusted_probability'] = df.apply(
    lambda row: row['probability'] if row['prediction'] == 1 else 1 - row['probability'], axis=1
)

# Check the data types and summary statistics to understand the structure of the dataset
df.info()

# Summary statistics
summary_stats = df.describe()

summary_stats

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4663 entries, 0 to 4662
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subject               4663 non-null   object 
 1   session               4663 non-null   object 
 2   image_t2              4663 non-null   object 
 3   image_dwi             4663 non-null   object 
 4   image_adc             4663 non-null   object 
 5   dep                   4663 non-null   float64
 6   ED                    4663 non-null   float64
 7   AF                    54 non-null     float64
 8   TB                    2223 non-null   float64
 9   TR                    15 non-null     float64
 10  PSA                   4271 non-null   float64
 11  VP                    2907 non-null   float64
 12  PIR                   1657 non-null   float64
 13  csPC                  4663 non-null   float64
 14  F_nacimiento          4663 non-null   object 
 15  F_RM                 

,dep,ED,AF,TB,TR,PSA,VP,PIR,csPC,VP_segmentation,prediction,probability,adjusted_probability
count,4663.000000,4663.00000,54.0,2223.000000,15.000000,4271.000000,2907.000000,1657.000000,4663.000000,217.000000,4663.000000,4663.000000,4663.000000
mean,12.320824,67.20609,11.0,1.730994,0.800000,9.732667,309.934372,3.841279,0.556294,52.302229,0.558868,0.893164,0.579665
std,5.950497,7.63514,0.0,0.942815,0.414039,19.418702,4876.350533,1.170092,0.496874,32.433966,0.496576,0.129413,0.406215
min,3.000000,36.00000,11.0,1.000000,0.000000,0.006000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500069,0.002726
25%,7.000000,62.00000,11.0,1.000000,1.000000,4.540000,31.000000,3.000000,0.000000,31.626270,0.000000,0.843725,0.121205
50%,17.000000,68.00000,11.0,1.000000,1.000000,6.980000,51.376000,4.000000,1.000000,44.452148,1.000000,0.954487,0.746560
75%,17.000000,73.00000,11.0,2.000000,1.000000,10.840000,77.168000,5.000000,1.000000,67.868848,1.000000,0.986816,0.984028
max,21.000000,96.00000,11.0,6.000000,1.000000,982.000000,116829.440000,5.000000,1.000000,149.049316,1.000000,0.998963,0.998963


In [102]:

# Drop rows with missing values in the columns of interest for the statistical analysis
df_clean = df.dropna(subset=['csPC', 'prediction', 'probability'])

# Convert to appropriate types if necessary
df_clean['csPC'] = df_clean['csPC'].astype(int)
df_clean['prediction'] = df_clean['prediction'].astype(int)

# Calculate performance metrics
accuracy = accuracy_score(df_clean['csPC'], df_clean['prediction'])
precision = precision_score(df_clean['csPC'], df_clean['prediction'])
recall = recall_score(df_clean['csPC'], df_clean['prediction'])
f1 = f1_score(df_clean['csPC'], df_clean['prediction'])

# Summary of performance metrics
performance_metrics = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# Calculate correlation between probability and csPC
correlation = df_clean['probability'].corr(df_clean['csPC'])

# Summary statistics for probability based on csPC classification
probability_stats = df_clean.groupby('csPC')['probability'].describe()


performance_metrics, correlation, probability_stats

({'Accuracy': 0.9090714132532705,
  'Precision': 0.9163468917881811,
  'Recall': 0.920585967617579,
  'F1 Score': 0.9184615384615384},
 0.2663362914452739,
        count      mean       std       min       25%       50%       75%  \
 csPC                                                                       
 0     2069.0  0.854575  0.135802  0.500069  0.772966  0.906623  0.966786   
 1     2594.0  0.923943  0.115181  0.500070  0.915049  0.980145  0.991835   
 
            max  
 csPC            
 0     0.998310  
 1     0.998963  )

In [103]:
import scipy.stats as stats

# Perform ANOVA to test if there is a significant difference in prediction probabilities across different health departments (dep)
anova_result = stats.f_oneway(*(df[df['dep'] == dep]['prediction'] for dep in df['dep'].unique()))

anova_result

F_onewayResult(statistic=49.34813196674221, pvalue=8.014122123025531e-86)

In [104]:
# Calculate efficiency (accuracy) for each PI-RADS group based on csPC labels
pir_groups = df['PIR'].unique()
efficiency_results = {}

for group in pir_groups:
    if not np.isnan(group) and group != 0:
        group_data = df[df['PIR'] == group]
        print(group_data.shape)
        accuracy = accuracy_score(group_data['csPC'], group_data['prediction'])
        efficiency_results[group] = accuracy

# Convert the results into a DataFrame for better readability
efficiency_df = pd.DataFrame.from_dict(efficiency_results, orient='index', columns=['Efficiency']).sort_index()

efficiency_df

(584, 22)
(106, 22)
(282, 22)
(572, 22)
(111, 22)


,Efficiency
1.0,0.882883
2.0,0.877358
3.0,0.854610
4.0,0.835616
5.0,0.884615


In [105]:
csPC_by_dep = df.groupby('dep')['csPC'].value_counts().unstack().fillna(0)

csPC_by_dep

csPC,0.0,1.0
dep,,
3.0,2.0,0.0
4.0,35.0,21.0
5.0,600.0,290.0
7.0,576.0,640.0
8.0,0.0,8.0
16.0,3.0,0.0
17.0,658.0,1318.0
19.0,16.0,25.0
20.0,81.0,33.0


In [106]:
# Get the counts of samples for each department
dep_counts = df['dep'].value_counts()

# Identify departments with more than 500 samples
large_deps = dep_counts[dep_counts > 100].index

# Filter the dataframe to include only departments with more than 500 samples
df_large_deps = df[df['dep'].isin(large_deps)]

# Calculate efficiency (accuracy) for each health department based on csPC labels
efficiency_by_large_dep = {}

for dep in large_deps:
    dep_data = df_large_deps[df_large_deps['dep'] == dep]
    accuracy = accuracy_score(dep_data['csPC'], dep_data['prediction'])
    efficiency_by_large_dep[dep] = accuracy

# Convert the results into a DataFrame for better readability
efficiency_by_large_dep_df = pd.DataFrame.from_dict(efficiency_by_large_dep, orient='index', columns=['Efficiency']).sort_index()

# Perform ANOVA to test if there is a significant difference in efficiencies across these large health departments
anova_large_deps_result = stats.f_oneway(*(df_large_deps[df_large_deps['dep'] == dep]['adjusted_probability'] for dep in large_deps))


efficiency_by_large_dep_df, anova_large_deps_result

(      Efficiency
 5.0     0.873034
 7.0     0.856086
 17.0    0.949899
 20.0    0.929825
 21.0    0.938375,
 F_onewayResult(statistic=134.71129563916523, pvalue=6.358978031596122e-109))

In [170]:
# Combine the overall dataset with department-specific data for comparison
df_total = df.copy()
df_total['dep'] = 'Total'

# Concatenate the total dataset with the large departments dataset
df_comparison = pd.concat([df_total, df_large_deps])

# Create the boxplot using Plotly
fig = px.box(
    df_comparison, 
    x='dep', 
    y='adjusted_probability', 
    color='dep',
    title='Probability Distribution Comparison Across Health Departments',

)


# Update layout to match the original style
fig.update_layout(
    xaxis_title='Health Department (dep)',
    yaxis_title='Adjusted Probability',
    xaxis_tickangle=45,
    width=1400,
    height=800,
    template='plotly_white',
    # plot_bgcolor='white',
    # paper_bgcolor='white',
    font=dict(color='black'),
    yaxis=dict(gridcolor='rgba(128,128,128,0.5)'),
)

# Display the plot
fig.show()

In [158]:

# Create a crosstab for ground truth vs prediction by dep
crosstab = pd.crosstab(index=[df_large_deps['dep'], df_large_deps['csPC']], columns=df_large_deps['prediction'])

# Reset the index for easier plotting
crosstab = crosstab.reset_index()

# Melt the dataframe to long format for seaborn plotting
crosstab_melted = crosstab.melt(id_vars=['dep', 'csPC'], value_vars=[0.0, 1.0], var_name='Prediction', value_name='Count')

# Create a new column for ground truth vs prediction comparison
crosstab_melted['Ground Truth vs Prediction'] = crosstab_melted.apply(lambda row: f'Truth: {int(row["csPC"])} | Pred: {int(row["Prediction"])}', axis=1)

# Define the traces for the plot
trace1 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['dep']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['Count'],
    name='Truth: 0 | Pred: 0',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['Count'],
    marker_color=colors[0]
)

trace2 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['dep']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['Count'],
    name='Truth: 1 | Pred: 0',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['Count'],
    marker_color=colors[1]
)

trace3 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['dep']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['Count'],
    name='Truth: 0 | Pred: 1',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['Count'],
    marker_color=colors[2]
)

trace4 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['dep']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['Count'],
    name='Truth: 1 | Pred: 1',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['Count'],
    marker_color=colors[3]
)

# Combine the traces
data = [trace1, trace2, trace3, trace4]

# Define the layout
layout = go.Layout(
    title='Ground Truth vs Prediction by Health Department',
    xaxis=dict(title='Health Department (dep)'),
    barmode='group',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black'),
    yaxis=dict(title='Count',gridcolor='rgba(128,128,128,0.5)'),
    legend=dict(title='Ground Truth vs Prediction')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Display the plot
fig.show()

In [110]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(df_large_deps['dep'], df_large_deps['prediction'])

# Perform the Chi-Square test
chi2, p, dof, ex = chi2_contingency(contingency_table)

chi2, p

(388.58037407664045, 8.156884309338058e-83)

In [114]:
from sklearn.metrics import confusion_matrix

# Function to calculate confusion matrix metrics for each department
def confusion_matrix_by_department(df):
    departments = df['dep'].unique()
    confusion_matrices = {}
    for dep in departments:
        dep_data = df[df['dep'] == dep]
        cm = confusion_matrix(dep_data['csPC'], dep_data['prediction'])
        tn, fp, fn, tp = cm.ravel()
        confusion_matrices[dep] = {
            'True Negative': tn,
            'False Positive': fp,
            'False Negative': fn,
            'True Positive': tp,
            'False Positive Rate': fp / (fp + tn),
            'False Negative Rate': fn / (fn + tp)
        }
    return confusion_matrices

confusion_matrices = confusion_matrix_by_department(df_large_deps)
confusion_matrices_df = pd.DataFrame.from_dict(confusion_matrices, orient='index')



confusion_matrices_df

,True Negative,False Positive,False Negative,True Positive,False Positive Rate,False Negative Rate
17.0,590,68,31,1287,0.103343,0.023520
5.0,533,67,46,244,0.111667,0.158621
20.0,75,6,2,31,0.074074,0.060606
7.0,516,60,115,525,0.104167,0.179688
21.0,85,13,9,250,0.132653,0.034749


In [137]:
# Function to calculate fairness metrics for each department
def fairness_metrics_with_actual_rate(df):
    departments = df['dep'].unique()
    metrics = {}
    for dep in departments:
        dep_data = df[df['dep'] == dep]
        prediction_rate = dep_data['prediction'].mean()
        actual_rate = dep_data['csPC'].mean()
        cm = confusion_matrix(dep_data['csPC'], dep_data['prediction'])
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
        else:
            tn = cm[0, 0] if cm.shape[0] > 0 and cm.shape[1] > 0 else 0
            fp = cm[0, 1] if cm.shape[0] > 0 and cm.shape[1] > 1 else 0
            fn = cm[1, 0] if cm.shape[0] > 1 and cm.shape[1] > 0 else 0
            tp = cm[1, 1] if cm.shape[0] > 1 and cm.shape[1] > 1 else 0
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        metrics[dep] = {
            'Prediction Rate': prediction_rate,
            'Actual Rate': actual_rate,
            'True Positive Rate': tpr,
            'False Positive Rate': fpr
        }
    return metrics

# Calculate fairness metrics for departments with more than 100 samples
fairness_metrics_large_100_with_actual_rate = fairness_metrics_with_actual_rate(df_large_deps)
fairness_metrics_df_large_100_with_actual_rate = pd.DataFrame.from_dict(fairness_metrics_large_100_with_actual_rate, orient='index')

#Shwing only 3 decimals
fairness_metrics_df_large_100_with_actual_rate['Prediction Rate'] = fairness_metrics_df_large_100_with_actual_rate['Prediction Rate'].apply(lambda x: round(x, 3))
fairness_metrics_df_large_100_with_actual_rate['Actual Rate'] = fairness_metrics_df_large_100_with_actual_rate['Actual Rate'].apply(lambda x: round(x, 3))

# Create the bar plot for prediction rate compared with the actual rate using Plotly
fig = go.Figure()

# Add prediction rate bars
fig.add_trace(go.Bar(
    x=[str(v) for v in fairness_metrics_df_large_100_with_actual_rate.index.values],
    y=fairness_metrics_df_large_100_with_actual_rate['Prediction Rate'],
    name='Prediction Rate',
    text=fairness_metrics_df_large_100_with_actual_rate['Prediction Rate'],
    textposition='auto',
    marker_color=colors[0]  # First color
))

# Add actual rate bars
fig.add_trace(go.Bar(
    x=[str(v) for v in fairness_metrics_df_large_100_with_actual_rate.index.values],
    y=fairness_metrics_df_large_100_with_actual_rate['Actual Rate'],
    name='Actual Rate',
    text=fairness_metrics_df_large_100_with_actual_rate['Actual Rate'],
    textposition='auto',
    marker_color=colors[1]  # Second color
))

# Customize the layout
fig.update_layout(
    title='Prediction Rate vs Actual Rate by Health Department',
    xaxis_title='Health Department (dep)',
    yaxis_title='Rate',
    barmode='group',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black'),
    legend=dict(
        title='Rate',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    yaxis=dict(gridcolor='rgba(128,128,128,0.5)')
)

# Show the plot
fig.show()

fairness_metrics_df_large_100_with_actual_rate

,Prediction Rate,Actual Rate,True Positive Rate,False Positive Rate
17.0,0.686,0.667,0.976480,0.103343
5.0,0.349,0.326,0.841379,0.111667
20.0,0.325,0.289,0.939394,0.074074
7.0,0.481,0.526,0.820312,0.104167
21.0,0.737,0.725,0.965251,0.132653


## Checking manufacturer

In [138]:

def confusion_matrix_and_fairness_by_manufacturer(df):
    manufacturers = df['Manufacturer'].unique()
    confusion_matrices = {}
    fairness_metrics = {}
    for manufacturer in manufacturers:
        manufacturer_data = df[df['Manufacturer'] == manufacturer]
        cm = confusion_matrix(manufacturer_data['csPC'], manufacturer_data['prediction'])
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
        else:
            tn = cm[0, 0] if cm.shape[0] > 0 and cm.shape[1] > 0 else 0
            fp = cm[0, 1] if cm.shape[0] > 0 and cm.shape[1] > 1 else 0
            fn = cm[1, 0] if cm.shape[0] > 1 and cm.shape[1] > 0 else 0
            tp = cm[1, 1] if cm.shape[0] > 1 and cm.shape[1] > 1 else 0
        prediction_rate = manufacturer_data['prediction'].mean()
        actual_rate = manufacturer_data['csPC'].mean()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        confusion_matrices[manufacturer] = {
            'True Negative': tn,
            'False Positive': fp,
            'False Negative': fn,
            'True Positive': tp,
            'False Positive Rate': fpr,
            'False Negative Rate': fn / (fn + tp) if (fn + tp) > 0 else 0
        }
        fairness_metrics[manufacturer] = {
            'Prediction Rate': prediction_rate,
            'Actual Rate': actual_rate,
            'True Positive Rate': tpr,
            'False Positive Rate': fpr
        }
    return confusion_matrices, fairness_metrics


In [139]:
#t2w df

df_t2w = pd.read_csv('Prostate_Cancer_TFM/Data_Analysis/Tables/t2w_df_complete.csv')

# Merge the datasets on common columns (assuming 'subject' and 'session' are the common columns)
df_merged = pd.merge(df_t2w, df, on=['subject', 'session'], how='inner')

# Create the Manufacturer column
df_merged['Manufacturer'] = df_merged['Manufacturer'].replace({
    'GE MEDICAL SYSTEMS': 'GE',
    'Philips Medical Systems': 'Philips',
    'SIEMENS': 'Siemens',
    'Siemens HealthCare GmbH': 'Siemens'
})


# Extract the unique subject-session pairs from the original dataframe
original_pairs_set = set(df[['subject', 'session']].apply(tuple, axis=1))

# Filter the merged dataframe to include only the subject and session pairs in the original dataframe
df_correct_filtered = df_merged[df_merged[['subject', 'session']].apply(tuple, axis=1).isin(original_pairs_set)]

# Ensure the merged dataframe only has one row per subject and session pair (drop duplicates)
df_correct_filtered_unique = df_correct_filtered.drop_duplicates(subset=['subject', 'session'])

# Add the Manufacturer column to the original dataframe
df_original_with_manufacturer = pd.merge(df, df_correct_filtered_unique[['subject', 'session', 'Manufacturer']], on=['subject', 'session'], how='left')



# Filter the merged dataframe to include only the relevant columns for analysis
df_filtered = df_original_with_manufacturer[['Manufacturer', 'csPC', 'prediction', 'dep']]





# Calculate confusion matrices and fairness metrics for each manufacturer
confusion_matrices_manufacturer, fairness_metrics_manufacturer = confusion_matrix_and_fairness_by_manufacturer(df_filtered)
confusion_matrices_df_manufacturer = pd.DataFrame.from_dict(confusion_matrices_manufacturer, orient='index')
fairness_metrics_df_manufacturer = pd.DataFrame.from_dict(fairness_metrics_manufacturer, orient='index')

confusion_matrices_df_manufacturer

,True Negative,False Positive,False Negative,True Positive,False Positive Rate,False Negative Rate
GE,1722,193,200,2321,0.100783,0.079334
Philips,98,11,2,35,0.100917,0.054054
Siemens,31,14,4,32,0.311111,0.111111


In [140]:
fairness_metrics_df_manufacturer

,Prediction Rate,Actual Rate,True Positive Rate,False Positive Rate
GE,0.566727,0.568305,0.920666,0.100783
Philips,0.315068,0.253425,0.945946,0.100917
Siemens,0.567901,0.444444,0.888889,0.311111


In [143]:
fig = go.Figure()

# Add prediction rate bars
fig.add_trace(go.Bar(
    x=fairness_metrics_df_manufacturer.index,
    y=fairness_metrics_df_manufacturer['Prediction Rate'],
    name='Prediction Rate',
    text=fairness_metrics_df_manufacturer['Prediction Rate'].apply(lambda x: round(x, 3)),
    textposition='auto',
    marker_color=colors[0]  # First color
))

# Add actual rate bars
fig.add_trace(go.Bar(
    x=fairness_metrics_df_manufacturer.index,
    y=fairness_metrics_df_manufacturer['Actual Rate'],
    name='Actual Rate',
    text=fairness_metrics_df_manufacturer['Actual Rate'].apply(lambda x: round(x, 3)),
    textposition='auto',
    marker_color=colors[1]  # Second color
))

# Customize the layout
fig.update_layout(
    title='Prediction Rate vs Actual Rate by Manufacturer',
    xaxis_title='Manufacturer',
    yaxis_title='Rate',
    barmode='group',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black'),
    legend=dict(
        title='Rate',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    yaxis=dict(gridcolor='rgba(128,128,128,0.5)')
)

# Show the plot
fig.show()

In [172]:
# Create a crosstab for ground truth vs prediction by dep
crosstab = pd.crosstab(index=[df_original_with_manufacturer['Manufacturer'], df_original_with_manufacturer['csPC']], columns=df_original_with_manufacturer['prediction'])

# Reset the index for easier plotting
crosstab = crosstab.reset_index()

# Melt the dataframe to long format for seaborn plotting
crosstab_melted = crosstab.melt(id_vars=['Manufacturer', 'csPC'], value_vars=[0.0, 1.0], var_name='Prediction', value_name='Count')

# Create a new column for ground truth vs prediction comparison
crosstab_melted['Ground Truth vs Prediction'] = crosstab_melted.apply(lambda row: f'Truth: {int(row["csPC"])} | Pred: {int(row["Prediction"])}', axis=1)

# Define the traces for the plot
trace1 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['Manufacturer']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['Count'],
    name='Truth: 0 | Pred: 0',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 0']['Count'],
    marker_color=colors[0]
)

trace2 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['Manufacturer']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['Count'],
    name='Truth: 1 | Pred: 0',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 0']['Count'],
    marker_color=colors[1]
)

trace3 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['Manufacturer']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['Count'],
    name='Truth: 0 | Pred: 1',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 0 | Pred: 1']['Count'],
    marker_color=colors[2]
)

trace4 = go.Bar(
    x=[str(v) for v in crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['Manufacturer']],
    y=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['Count'],
    name='Truth: 1 | Pred: 1',
    text=crosstab_melted[crosstab_melted['Ground Truth vs Prediction'] == 'Truth: 1 | Pred: 1']['Count'],
    marker_color=colors[3]
)

# Combine the traces
data = [trace1, trace2, trace3, trace4]

# Define the layout
layout = go.Layout(
    title='Ground Truth vs Prediction by Health Department',
    xaxis=dict(title='Health Department (dep)'),
    barmode='group',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black'),
    yaxis=dict(title='Count',gridcolor='rgba(128,128,128,0.5)'),
    legend=dict(title='Ground Truth vs Prediction')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Display the plot
fig.show()

In [80]:
from sklearn.metrics import classification_report

# Function to calculate classification report metrics by csPC and by each Manufacturer
def classification_report_by_csPC_and_manufacturer(df):
    manufacturers = df['Manufacturer'].unique()
    reports = {}
    for manufacturer in manufacturers:
        manufacturer_data = df[df['Manufacturer'] == manufacturer]
        report = classification_report(manufacturer_data['csPC'], manufacturer_data['prediction'], output_dict=True)
        reports[manufacturer] = report
    return reports

# Calculate classification report metrics by csPC and by each Manufacturer
classification_reports = classification_report_by_csPC_and_manufacturer(df_original_with_manufacturer)

# Convert the classification reports into a DataFrame for better readability
report_dfs = {}
for key, report in classification_reports.items():
    report_df = pd.DataFrame(report).transpose()
    report_df['Manufacturer'] = key
    report_dfs[key] = report_df

# Concatenate all DataFrames into one
final_report_df = pd.concat(report_dfs.values())

final_report_df

,precision,recall,f1-score,support,Manufacturer
0.0,0.895942,0.899217,0.897576,1915.000000,GE
1.0,0.923230,0.920666,0.921946,2521.000000,GE
accuracy,0.911407,0.911407,0.911407,0.911407,GE
macro avg,0.909586,0.909942,0.909761,4436.000000,GE
weighted avg,0.911450,0.911407,0.911426,4436.000000,GE
0.0,0.980000,0.899083,0.937799,109.000000,Philips
1.0,0.760870,0.945946,0.843373,37.000000,Philips
accuracy,0.910959,0.910959,0.910959,0.910959,Philips
macro avg,0.870435,0.922514,0.890586,146.000000,Philips
weighted avg,0.924467,0.910959,0.913869,146.000000,Philips


In [74]:
csPC_by_man = df_original_with_manufacturer.groupby('Manufacturer')['csPC'].value_counts().unstack().fillna(0)

csPC_by_man

csPC,0.0,1.0
Manufacturer,,
GE,1915,2521
Philips,109,37
Siemens,45,36


In [173]:
# Combine the overall dataset with department-specific data for comparison
df_total = df_original_with_manufacturer.copy()
df_total['Manufacturer'] = 'All Manufacturers'

# Concatenate the total dataset with the large departments dataset
df_comparison = pd.concat([df_total, df_original_with_manufacturer])

# Create the boxplot using Plotly
fig = px.box(
    df_comparison, 
    x='Manufacturer', 
    y='adjusted_probability', 
    color='Manufacturer',
    title='Probability Distribution Comparison Across Health Departments',

)


# Update layout to match the original style
fig.update_layout(
    xaxis_title='Manufacturer',
    yaxis_title='Adjusted Probability',
    xaxis_tickangle=45,
    width=1400,
    height=800,
    template='plotly_white',
    # plot_bgcolor='white',
    # paper_bgcolor='white',
    font=dict(color='black'),
    yaxis=dict(gridcolor='rgba(128,128,128,0.5)'),
)

# Display the plot
fig.show()